In [1]:
import os
import subprocess as sp
from iterextras import par_for
from multiprocessing import cpu_count
from tqdm.auto import tqdm

In [2]:
CRATES = [
    ('Rocket', 'core/lib/', ''),
    ('image', '', ''),
    ('rayon', '', None),
    ('rg3d', '', None),    
    ('nalgebra', '', '--features rand,arbitrary,sparse,debug,io,libm'), # avoid no_unsound_assume_init feature!
    ('rustls', 'rustls/', None),
    ('sccache', '', ''),
    ('hyper', '', '--features full'),    
    ('rav1e', '', ''),
    ('RustPython', 'vm/', '--features compiler'),
]

In [18]:
CLOC = '/home/wcrichto/.node_modules/bin/cloc'
total_loc = 0
for (name, subdir, _) in CRATES:
    cmd = f"{CLOC} {name}/{subdir}src --json | jq '.Rust.code'"
    outp = sp.check_output(cmd, shell=True, stderr=sp.STDOUT, cwd=os.path.abspath('../data/repos'))
    loc = int(outp.decode('utf-8').strip())
    total_loc += loc
    print(name, loc)
total_loc

Rocket 9313
image 19718
rayon 15524
rg3d 54426
nalgebra 31951
rustls 16866
sccache 23202
hyper 15082
rav1e 50294
RustPython 48124


284500

In [ ]:
#NUM_THREADS = cpu_count()
NUM_THREADS = 30
CLEAN = False

def analyze_crate(crate):
    (name, subdir, feature_flags) = crate
    lib_path = os.path.abspath(f'../data/repos/{name}/{subdir}src/lib.rs')
    out_path = os.path.abspath(f'../data/{name.replace("/", "_")}.json')
    working_dir = f'../data/repos/{name}/{subdir}'
    
    if os.path.exists(out_path) and os.stat(out_path).st_size > 0:
        return
    
    dyld_lib_path = 'LD_LIBRARY_PATH=$($(rustup which --toolchain local rustc) --print target-libdir):$LD_LIBRARY_PATH'
    feature_flags = feature_flags if feature_flags is not None else "--all-features"
    cargo_cmd = f'cargo check {feature_flags}'
    if CLEAN:
        cargo_cmd = f'cargo clean && {cargo_cmd}'        
    cmd = f'rm -f {out_path} && rustup override set local && {cargo_cmd} && {dyld_lib_path} RUST_BACKTRACE=1 RUST_LOG="rust_slicer_eval=debug" rust-slicer-eval -j {NUM_THREADS} {lib_path} {feature_flags} {out_path}'
    try:
#         print(cmd)
        sp.check_output(cmd, shell=True, stderr=sp.PIPE, cwd=working_dir)
    except sp.CalledProcessError as e:
        print(f'Stderr for {name}')
        print(e.stderr.decode('utf-8'))

In [ ]:
# for crate in tqdm(CRATES):
#     analyze_crate(crate)
par_for(analyze_crate, CRATES, workers=2)